In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class VisibilityDataset(Dataset):
    def __init__(self, obstacle_file, query_file, dim):
        """
        Args:
            obstacle_file (str): Path to the file containing obstacle encodings (dim*dim grids).
            query_file (str): Path to the file containing queries (x1, y1, x2, y2, label).
            dim (int): Dimension of the obstacle grid.
        """
        # Load the obstacle grids
        self.obstacle_data = self.load_obstacles(obstacle_file, dim)
        # Load the queries (x1, y1, x2, y2, label)
        self.query_data = self.load_queries(query_file)
        
    def load_obstacles(self, file_path, dim):
        """
        Loads obstacle data from a file into memory.
        Args:
            file_path (str): Path to the obstacle file.
            dim (int): Dimension of the grid.
        Returns:
            List of numpy arrays (dim, dim) representing the obstacle encodings.
        """
        obstacles = []
        with open(file_path, 'r') as file:
            # Read dim lines to construct the dim*dim grid for each scene
            for _ in range(dim):
                line = list(map(float, file.readline().strip().split()))  # Read each line and split into elements
                obstacles.append(line)  # Stack all lines to form the grid
        return np.array(obstacles, dtype=np.float32)
    
    def load_queries(self, file_path):
        """
        Loads query data from a file into memory.
        Args:
            file_path (str): Path to the query file.
        Returns:
            List of tuples (x1, y1, x2, y2, label).
        """
        queries = []
        with open(file_path, 'r') as file:
            for line in file:
                x1, y1, x2, y2, label = map(float, line.split())
                queries.append((x1, y1, x2, y2, label))
        return np.array(queries)
    
    def __len__(self):
        return len(self.query_data)
    
    def __getitem__(self, idx):
        """
        Get a sample from the dataset.
        Args:
            idx (int): Index of the sample to fetch.
        Returns:
            A concatenated 1D tensor of obstacle grid and query.
        """
        # Flatten the obstacle grid and convert it to tensor
        obstacle_grid = torch.tensor(self.obstacle_data.flatten(), dtype=torch.float32)
        
        # Get the query and convert to tensor
        query = torch.tensor(self.query_data[idx][:4], dtype=torch.float32)  # x1, y1, x2, y2
        label = torch.tensor(self.query_data[idx][4], dtype=torch.float32)  # Label
        
        # Concatenate obstacle grid and query in one dimension
        input = torch.cat((obstacle_grid, query))
        
        return input, label

dim = 64

# Initialize the dataset
train_dataset = VisibilityDataset("encoding1_64x64.txt", "query_64x64.txt", dim)

# Create the DataLoader with batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)


In [2]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix


# Define helper function to compute metrics
def get_metrics(y_true, y_pred):
    y_pred = y_pred.round()  # Convert probabilities to binary predictions (0 or 1)
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, zero_division=0)
    precision = precision_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    specificity = tn / (tn + fp) if tn + fp > 0 else 0  # Specificity = TN / (TN + FP)
    
    return accuracy, recall, precision, f1, specificity

# Define the BinaryClassificationModel
class BinaryClassificationModel(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassificationModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
            nn.Sigmoid()  # Output between 0 and 1 for binary classification
        )

    def forward(self, x):
        return self.model(x)

# Evaluation function to get metrics on validation or test set
def evaluate(model, data_loader, criterion, device):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_preds = []
    total_loss = 0  # To accumulate the loss during evaluation

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move to device (GPU if available)
            outputs = model(inputs)

            # Calculate the loss
            loss = criterion(outputs.squeeze(), labels.float())
            total_loss += loss.item()

            all_labels.append(labels.cpu().numpy())
            all_preds.append(outputs.cpu().numpy())

    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)

    # Compute metrics over the entire dataset
    accuracy, recall, precision, f1, specificity = get_metrics(all_labels, all_preds)
    
    # Calculate the average loss
    avg_loss = total_loss / len(data_loader)
    
    return avg_loss, accuracy, recall, precision, f1, specificity

# Helper function to train and evaluate the model
def train(model, train_loader, val_loader, num_epochs, lr=1e-3):
    
    model.to(device)  # Move the model to GPU/CPU
    
    # Loss function and optimizer
    criterion = nn.BCELoss()  # Binary Cross-Entropy loss for binary classification
    optimizer = optim.Adam(model.parameters(), lr=lr)

    best_val_f1 = 0  # To track the best model based on validation F1 score

    for epoch in range(num_epochs):
        # Training phase
        model.train()  # Set model to training mode
        train_loss = 0
        train_metrics = {"accuracy": [], "recall": [], "precision": [], "f1": [], "specificity": []}

        with tqdm(train_loader, desc="Training", unit="batch") as pbar:
            for inputs, labels in pbar:
                inputs, labels = inputs.to(device), labels.to(device)  # Move to device (GPU if available)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), labels.float())
                loss.backward()
                optimizer.step()

                # Calculate metrics for the batch
                outputs = outputs.detach().cpu().numpy()
                labels = labels.detach().cpu().numpy()
                acc, rec, prec, f1, spec = get_metrics(labels, outputs)
                
                train_loss += loss.item()
                train_metrics["accuracy"].append(acc)
                train_metrics["recall"].append(rec)
                train_metrics["precision"].append(prec)
                train_metrics["f1"].append(f1)
                train_metrics["specificity"].append(spec)

                # Update the tqdm progress bar with all metrics
                pbar.set_postfix(
                    loss=train_loss / (pbar.n + 1),
                    accuracy=np.mean(train_metrics["accuracy"]),
                    recall=np.mean(train_metrics["recall"]),
                    precision=np.mean(train_metrics["precision"]),
                    f1=np.mean(train_metrics["f1"]),
                    specificity=np.mean(train_metrics["specificity"])
                )
            
        # Evaluation phase (for training metrics)
        train_loss, train_accuracy, train_recall, train_precision, train_f1, train_specificity = \
            evaluate(model, train_loader, criterion, device)
        
        # Evaluation phase (for validation metrics)
        val_loss, val_accuracy, val_recall, val_precision, val_f1, val_specificity = \
            evaluate(model, val_loader, criterion, device)

        # Display the per-epoch training and validation results
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Training - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Recall: {train_recall:.4f}, Precision: {train_precision:.4f}, F1: {train_f1:.4f}, Specificity: {train_specificity:.4f}")
        print(f"Validation - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Recall: {val_recall:.4f}, Precision: {val_precision:.4f}, F1: {val_f1:.4f}, Specificity: {val_specificity:.4f}")

        # Save the best model based on validation F1 score
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), 'best_model.pth')

# Check if GPU is available, otherwise fallback to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


model = BinaryClassificationModel(dim*dim+4)

# Train and evaluate the model
train(model, train_loader, val_loader, num_epochs=100, lr=1e-3)

# model.load_state_dict(torch.load('best_model.pth', weights_only=True))

print("\nEvaluating on Test Set:")
test_loss, test_accuracy, test_recall, test_precision, test_f1, test_specificity = \
    evaluate(model, test_loader, nn.BCELoss(), device)

# Print all metrics for the test set
print(f"Test - Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}, Recall: {test_recall:.4f}, Precision: {test_precision:.4f}, F1: {test_f1:.4f}, Specificity: {test_specificity:.4f}")



Using device: cpu


Training: 100%|██████████| 1/1 [00:00<00:00, 15.26batch/s, accuracy=0.62, f1=0.513, loss=0.686, precision=0.556, recall=0.476, specificity=0.724]


Epoch 1/100
Training - Loss: 0.7362, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000
Validation - Loss: 0.7362, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000


Training: 100%|██████████| 1/1 [00:00<00:00, 18.59batch/s, accuracy=0.58, f1=0, loss=0.836, precision=0, recall=0, specificity=1]


Epoch 2/100
Training - Loss: 0.6931, Accuracy: 0.6400, Recall: 0.6190, Precision: 0.5652, F1: 0.5909, Specificity: 0.6552
Validation - Loss: 0.6931, Accuracy: 0.6400, Recall: 0.6190, Precision: 0.5652, F1: 0.5909, Specificity: 0.6552


...

Epoch 98/100
Training - Loss: 0.6830, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000
Validation - Loss: 0.6830, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000


Training: 100%|██████████| 1/1 [00:00<00:00, 21.42batch/s, accuracy=0.52, f1=0.143, loss=0.696, precision=0.286, recall=0.0952, specificity=0.828]


Epoch 99/100
Training - Loss: 0.6839, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000
Validation - Loss: 0.6839, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000


Training: 100%|██████████| 1/1 [00:00<00:00, 15.05batch/s, accuracy=0.64, f1=0.25, loss=0.683, precision=1, recall=0.143, specificity=1]


Epoch 100/100
Training - Loss: 0.6846, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000
Validation - Loss: 0.6846, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000

Evaluating on Test Set:
Test - Loss: 0.6846, Accuracy: 0.5800, Recall: 0.0000, Precision: 0.0000, F1: 0.0000, Specificity: 1.0000
